In [15]:
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, BatchNormalization
from keras.layers import LeakyReLU, Dropout, Lambda, Input, Activation
from keras.optimizers import Adam
from keras.models import Model, Sequential
import keras.backend as K

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
print('Shapes :-')
print('x_train;', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

Shapes :-
x_train; (60000, 28, 28)
y_train: (60000,)
x_test: (10000, 28, 28)
y_test: (10000,)


In [5]:
x_train = x_train.astype('float32') / 255.
x_train = np.expand_dims(x_train, -1)

x_test = x_test.astype('float32') / 255.
x_test = np.expand_dims(x_test, -1)

In [6]:
print('Shapes :-')
print('x_train;', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

Shapes :-
x_train; (60000, 28, 28, 1)
y_train: (60000,)
x_test: (10000, 28, 28, 1)
y_test: (10000,)


Encoder and Decoder Definition for Variational Autoencoder

In [11]:
encoder_num_filters = [32, 64, 64, 64]
encoder_kernel_sizes = [3, 3, 3, 3]
encoder_strides = [1, 2, 2, 1]

decoder_num_filters = [64, 64, 32, 1]
decoder_kernel_sizes = [3, 3, 3, 3]
decoder_strides = [1, 2, 2, 1]

batch_normalization = True
dropout = True

latent_dim = 2

In [12]:
encoder_input = Input(shape=x_train.shape[1:], name='encoder_input')
x = encoder_input

for filter_num, kernel_size, stride in zip(encoder_num_filters, encoder_kernel_sizes, encoder_strides):
    conv_layer = Conv2D(filters=filter_num,
                        kernel_size=kernel_size,
                        strides=stride,
                        padding='same')
    x = conv_layer(x)
    if batch_normalization:
        x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    if dropout:
        x = Dropout(rate=0.25)(x)

shape_before_flatten = K.int_shape(x)[1:]
x = Flatten()(x)

mu = Dense(latent_dim, name='mu')(x)
log_var = Dense(latent_dim, name='log_var')(x)

# This will be useful for analyzing our model later
encoder_mu_log_var = Model(encoder_input, (mu, log_var))

def sampling(args):
    mu, log_var = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
    return mu + K.exp(log_var/2) * epsilon

encoder_output = Lambda(sampling, name='encoder_output')([mu, log_var])
encoder = Model(encoder_input, encoder_output)

W0903 21:50:29.802012 140666508252992 deprecation_wrapper.py:119] From /home/vaibhav/projects/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 21:50:29.803824 140666508252992 deprecation_wrapper.py:119] From /home/vaibhav/projects/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 21:50:29.854305 140666508252992 deprecation_wrapper.py:119] From /home/vaibhav/projects/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0903 21:50:29.855887 140666508252992 deprecation_wrapper.py:119] From /home/vaibhav/projects/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.Confi

In [13]:
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

In [18]:
decoder_input = Input(shape=(latent_dim, ), name='decoder_input')
x = Dense( np.prod(shape_before_flatten) )(decoder_input)
x = Reshape( shape_before_flatten )(x)

for i, (filter_num, kernel_size, stride) in enumerate(zip(decoder_num_filters, decoder_kernel_sizes, decoder_strides)):
    deconv_layer = Conv2DTranspose(filters=filter_num,
                                   kernel_size=kernel_size,
                                   strides=stride,
                                   padding='same')
    x = deconv_layer(x)
    if batch_normalization:
        x = BatchNormalization()(x)
    if i < len(decoder_kernel_sizes) - 1:
        x = LeakyReLU()(x)
    else:
        x = Activation('sigmoid')(x)
    if dropout:
        x = Dropout(rate=0.25)(x)

decoder_output = x
decoder = Model(decoder_input, decoder_output)

In [19]:
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_4 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 7, 7, 64)          36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 64)          0         
__________

In [20]:
autoencoder_input = encoder_input
autoencoder_output = decoder( encoder(autoencoder_input) )

autoencoder = Model(autoencoder_input, autoencoder_output)

In [21]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 28, 28, 1)         0         
_________________________________________________________________
model_2 (Model)              (None, 2)                 106116    
_________________________________________________________________
model_3 (Model)              (None, 28, 28, 1)         102661    
Total params: 208,777
Trainable params: 208,007
Non-trainable params: 770
_________________________________________________________________
